<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Gemini_2_5_Flash_Flight_Planner_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Environment

In [1]:
## libraries  --- Installation ---

!pip install flask -q
!pip install flask_migrate -q
!pip install flask_sqlalchemy -q
!pip install anthropic -q
!pip install colab-env --quiet
!pip install flask_cors -q

!pip install flask_ngrok -q
!pip install pyngrok -q  # Install pyngrok directly
!pip install --upgrade flask_ngrok pyngrok -q
!pip install flask_login -q

!pip install Flask-Login -q

!pip install flask_login -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install colab_env -q

## Gemini 2.5 Flash Flight Planner

In [3]:
import os
import socket
import threading
import time
import json
import logging # Import logging module

import warnings
warnings.filterwarnings("ignore")

from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import google.generativeai as genai

# 1. Flask Application Definition
app = Flask(__name__)

# Suppress the Werkzeug local proxy warnings for cleaner logs
# This targets the 'werkzeug.local' logger and sets its level to CRITICAL,
# effectively silencing most messages, including the "Working outside of request context" errors

#logging.getLogger('werkzeug.local').setLevel(logging.CRITICAL)
#print("Werkzeug local proxy warnings suppressed for cleaner logs.")


# Optional: Set a secret key if you plan to use sessions (highly recommended for production apps)
# app.secret_key = os.getenv('FLASK_SECRET_KEY', 'your_super_secret_key_here')

# 2. Configuration for Agent
class AgentConfig:
    LLM_MODEL_NAME: str = "gemini-2.5-flash"

# 3. Google Colab / Gemini API Imports and Configuration
GOOGLE_API_KEY = None
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GEMINI')
    print("Google Generative AI configured successfully using Colab Secrets.")
except (ImportError, KeyError):
    print("Not running in Google Colab or 'GEMINI' secret not found. Attempting to get 'GEMINI' environment variable.")
    GOOGLE_API_KEY = os.getenv('GEMINI')

if not GOOGLE_API_KEY:
    print("WARNING: GEMINI API Key not found. Gemini API calls may fail.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)

# 4. Define the HTML content as a Python string.
HTML_CONTENT = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Gemini 2.5 Flash Flight Planner AI</title>
    <style>
        body { font-family: sans-serif; margin: 2em; background-color: #f4f4f4; }
        .container { background-color: white; padding: 2em; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); max-width: 800px; margin: auto; }
        h1 { text-align: center; color: #333; }
        label { display: block; margin-top: 1em; font-weight: bold; }
        input[type="text"], input[type="date"], input[type="number"], select, textarea {
            width: calc(100% - 22px);
            padding: 10px;
            margin-top: 0.5em;
            border: 1px solid #ddd;
            border-radius: 4px;
            box-sizing: border-box;
        }
        textarea { resize: vertical; min-height: 80px; }
        .form-row { display: flex; justify-content: space-between; gap: 20px; }
        .form-row > div { flex: 1; }
        button {
            background-color: #4CAF50;
            color: white;
            padding: 12px 20px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 1em;
            margin-top: 1.5em;
            width: 100%;
        }
        button:hover { background-color: #45a049; }
        #loadingMessage {
            text-align: center;
            margin-top: 1em;
            color: #555;
            font-style: italic;
            display: none;
        }
        #generatedPlan {
            margin-top: 2em;
            padding: 1.5em;
            background-color: #e9ecef;
            border-radius: 8px;
            white-space: pre-wrap; /* Preserves formatting from AI response */
            word-wrap: break-word;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>✈️ Gemini 2.5 Flash Flight Planner AI ✈️</h1>
        <p>Input your flight details to get an optimal plan:</p>

        <form id="flightPlanForm">
            <div class="form-row">
                <div>
                    <label for="origin">Origin (IATA Code):</label>
                    <input type="text" id="origin" name="origin" value="YUL" required>
                </div>
                <div>
                    <label for="destination">Destination (IATA Code):</label>
                    <input type="text" id="destination" name="destination" value="ATL" required>
                </div>
            </div>

            <div class="form-row">
                <div>
                    <label for="departureDate">Departure Date:</label>
                    <input type="date" id="departureDate" name="departureDate" value="2025-06-30" required>
                </div>
                <div>
                    <label for="returnDate">Return Date (Optional):</label>
                    <input type="date" id="returnDate" name="returnDate" value="2025-07-08">
                </div>
            </div>

            <div class="form-row">
                <div>
                    <label for="passengers">Passengers:</label>
                    <input type="number" id="passengers" name="passengers" value="1" min="1" required>
                </div>
                <div>
                    <label for="travelClass">Travel Class:</label>
                    <select id="travelClass" name="travelClass">
                        <option value="Economy">Economy</option>
                        <option value="Premium Economy">Premium Economy</option>
                        <option value="Business">Business</option>
                        <option value="First Class">First Class</option>
                    </select>
                </div>
            </div>

            <label for="preferences">Additional Preferences (Optional):</label>
            <textarea id="preferences" name="preferences" placeholder="e.g., specific airline, direct flights only, window seat preference">DELTA</textarea>

            <button type="submit">Plan Flight</button>
        </form>

        <div id="loadingMessage">Generating flight plan...</div>
        <div id="generatedPlan">
            <p>Generated Flight Plan:</p>
            <p>Optimal Delta flight plan from YUL to ATL and return on specified dates, prioritizing direct flights and minimizing travel time.
Flight DL1234 (Delta)
Route: YUL to ATL
Depart: 2025-06-30T10:00:00 | Arrive: 2025-06-30T12:00:00
Duration: 2h00m

Flight DL5678 (Delta)
Route: ATL to YUL
Depart: 2025-07-08T13:00:00 | Arrive: 2025-07-08T15:00:00
Duration: 2h00m</p>
        </div>
    </div>

    <script>
        document.getElementById('flightPlanForm').addEventListener('submit', async function(event) {
            event.preventDefault(); // Prevent default form submission

            const loadingMessage = document.getElementById('loadingMessage');
            const generatedPlanDiv = document.getElementById('generatedPlan');
            loadingMessage.style.display = 'block'; // Show loading message
            generatedPlanDiv.innerHTML = '<p>Generated Flight Plan:</p>'; // Clear previous content

            const origin = document.getElementById('origin').value;
            const destination = document.getElementById('destination').value;
            const departureDate = document.getElementById('departureDate').value;
            const returnDate = document.getElementById('returnDate').value;
            const passengers = document.getElementById('passengers').value;
            const travelClass = document.getElementById('travelClass').value;
            const preferences = document.getElementById('preferences').value;

            // Construct the prompt for the AI
            let prompt = `Generate an optimal flight plan for a trip from ${origin} to ${destination}`;
            prompt += ` departing on ${departureDate}`;
            if (returnDate) {
                prompt += ` and returning on ${returnDate}`;
            }
            prompt += ` for ${passengers} passenger(s) in ${travelClass} class.`;
            if (preferences) {
                prompt += ` Additional preferences: ${preferences}.`;
            }
            // Ask for a structured response from the AI for better parsing/display
            prompt += ` Please provide the flight plan in a clear, readable format including:
            - A summary of the plan.
            - For each flight: Flight Number, Airline, Route (Origin to Destination), Departure Date/Time, Arrival Date/Time, Duration.`;

            try {
                const response = await fetch('/api/reason', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({ prompt: prompt })
                });

                const data = await response.json();

                loadingMessage.style.display = 'none'; // Hide loading message

                if (response.ok) {
                    generatedPlanDiv.innerHTML = '<p>Generated Flight Plan:</p>' + data.response;
                } else {
                    generatedPlanDiv.innerHTML = `<p>Error: ${data.error || 'Unknown error'}</p>`;
                    console.error('API Error:', data.error);
                }
            } catch (error) {
                loadingMessage.style.display = 'none'; // Hide loading message
                generatedPlanDiv.innerHTML = `<p>An error occurred: ${error.message}</p>`;
                console.error('Fetch Error:', error);
            }
        });
    </script>
</body>
</html>
"""

print("HTML content loaded.")

# 5. Flask Routes
@app.route('/')
def index():
    """Serves the main HTML content."""
    return render_template_string(HTML_CONTENT)

@app.route('/api/reason', methods=['POST'])
def reason():
    """
    Handles reasoning requests by sending the prompt to the Gemini 1.5 Flash model
    and returning its generated response.
    """
    if not GOOGLE_API_KEY:
        return jsonify({"error": "Gemini API key not configured on the backend."}), 500

    data = request.get_json()
    prompt = data.get('prompt', 'Generate a general AI response.')

    try:
        model = genai.GenerativeModel(AgentConfig.LLM_MODEL_NAME)
        response = model.generate_content(prompt)
        generated_text = ""

        # Ensure the model's response is converted to text
        if response.parts:
            generated_text = response.text
        elif response.candidates and len(response.candidates) > 0 and response.candidates[0].content:
            generated_text = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))
        else:
            generated_text = "No text content found in Gemini response."

        return jsonify({"response": generated_text})

    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        if "safety" in str(e).lower() or "blocked" in str(e).lower():
            return jsonify({"error": "AI response blocked due to safety concerns or content policy. Please try a different query."}), 500
        else:
            return jsonify({"error": f"Failed to get response from AI: {e}"}), 500

# 6. Helper function to find a free port
def find_free_port():
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('0.0.0.0', 0)) # Bind to port 0 to let OS find a free port
    port = sock.getsockname()[1]
    sock.close()
    return port

Google Generative AI configured successfully using Colab Secrets.
HTML content loaded.


In [ ]:
import colab_env

import os
import socket
import threading
import time
import json
import logging # Import logging module

import warnings
warnings.filterwarnings("ignore")

from flask import Flask, render_template_string, request, jsonify
from pyngrok import ngrok
import google.generativeai as genai

# 7. Application Startup
if __name__ == '__main__':
    # Ensure any previous ngrok tunnels are killed before starting a new one
    #ngrok.kill()

    try:
        # Get ngrok authtoken from environment variable
        #authtoken = None

        #try:
        #    from google.colab import userdata
        #    authtoken = userdata.get('NGROK_TOKEN')
        #except (ImportError, KeyError):
        #    authtoken = os.getenv('NGROK_TOKEN')



        # 1. Get authtoken from environment variable
        import colab_env
        authtoken = os.environ.get('NGROK_TOKEN')

        # 2. Check if authtoken is set
        if authtoken:
            ngrok.set_auth_token(authtoken)
        else:
            raise ValueError("NGROK_AUTHTOKEN environment variable not set. Please set it in Colab's environment variables or using os.environ.")

        #if not authtoken:
        #    raise ValueError("NGROK_TOKEN environment variable not set. Please set it in Colab's Secret.")

        ngrok.set_auth_token(authtoken)
        print("ngrok authtoken loaded.")

        # Find a free port for Flask to use
        #flask_port = find_free_port()
        #print(f"Flask will attempt to run on dynamically assigned port: {flask_port}")

        # Establish the ngrok tunnel to the dynamically assigned Flask port
        #print(f"Attempting to establish ngrok tunnel to port {flask_port}...")
        #public_url = ngrok.connect(flask_port).public_url


        # 3. Establish the ngrok tunnel
        public_url = ngrok.connect(5000).public_url
        #print(f"ngrok tunnel started at: {public_url}")


        # --- Run Flask app ---
        #app.run(host="0.0.0.0", port=5000, use_reloader=False, debug=True)

        print(f"ngrok tunnel started at: {public_url}")
        print("Click the link above to view the demo!")

        # Run Flask app in the main thread (blocking call)
        # Using debug=True to ensure proper error reporting and functionality.

        #print(f"Starting Flask app on port {flask_port} with debug True...")

        app.run(host="0.0.0.0", port=5000, use_reloader=False, debug=True)
        print("Flask app has stopped.")

    except Exception as e:
        print(f"An error occurred during setup or execution: {e}")
        print("Please ensure your ngrok authtoken (NGROK_TOKEN) and Gemini API key (GEMINI) are correct.")
    finally:
        ngrok.kill()
        print("ngrok tunnel disconnected.")

Mounted at /content/gdrive
ngrok authtoken loaded.
ngrok tunnel started at: https://7d07-34-48-182-77.ngrok-free.app
Click the link above to view the demo!
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Jun/2025 18:58:14] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Jun/2025 18:58:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [29/Jun/2025 18:59:01] "POST /api/reason HTTP/1.1" 200 -
